In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install octis

     |████████████████████████████████| 129 kB 8.5 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 16.3 MB 59.5 MB/s 
     |████████████████████████████████| 100 kB 11.9 MB/s 
     |████████████████████████████████| 22.3 MB 2.0 MB/s 
     |████████████████████████████████| 170 kB 60.5 MB/s 
     |████████████████████████████████| 24.1 MB 53.0 MB/s 
     |████████████████████████████████| 3.8 MB 14.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.8 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 45.2 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp37-cp37m-linux_x86_64.whl size=233366 sha256=fa9fbaad4bd069dc3b4d875e440252838dff2cef6ebb301359efdae4e576c7a4
  Stored in directory: /root/.cache/pip/wheels/cd/e8/1e/bf95cf256e4d3ffc94289ab508c

In [9]:
! pip install tweet-preprocessor

In [7]:
cd /content/drive/MyDrive/Topic Modelling with NMF

/content/drive/MyDrive/Topic Modelling with NMF


In [10]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
import spacy
# import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
import preprocessor as tweet_processor

In [11]:
import pandas as pd
# from stop_words import get_stop_words

In [12]:
data = pd.read_csv("Tweets.csv")

In [13]:
spacy_nlp = spacy.load("en_core_web_sm")
spacy_nlp.Defaults.stop_words |= {"o" ,"isn`t","ohhhh" ,",","."}
# spacy_nlp.Defaults.stop_words

In [14]:
def tokenizer(tweet):
    token = []
    clean_tweet = tweet_processor.clean(tweet)
    spacy_tweet = spacy_nlp(clean_tweet)
    
    
#     token = [word.lemma_.lower().split() for word in spacy_tweet if not word.is_stop and not word.is_punct and not word.like_num]
    for word in spacy_tweet:
        if not word.is_stop and not word.is_punct and not word.like_num:
            token += word.lemma_.split()
    return token

In [15]:
token = tokenizer('@USAirways  is there a better time to call? My flight is on Friday and I need to change it. Worried I may be on hold until then.')
token

['well', 'time', 'flight', 'Friday', 'need', 'change', 'worried', 'hold']

In [16]:
airline_data = data[data['airline']=='US Airways']

airline_tweets = airline_data['text'].tolist()
# airline_tweets

In [17]:
def format_NMF_output(nmf_H, nmf_W, vocab, number_top_words):
  topics = []
  for topic_idx, topic in enumerate(nmf_H):
    word_list = [vocab[i] for i in topic.argsort()[:-(number_top_words + 1):-1]]
    topics.append(word_list)

  octis_topic_dict = {}
  octis_topic_dict["topic-word-matrix"] = nmf_H
  octis_topic_dict["topics"] = topics
  octis_topic_dict["topic-document-matrix"] = np.array(nmf_W).transpose()
  return octis_topic_dict

In [18]:
def prep_dataset_for_octis(airline_tweets):
    dataset_for_octis = []
    for tweets in airline_tweets:
        token = tokenizer(tweets)
        dataset_for_octis.append(token)  
    return dataset_for_octis

In [19]:
number_topic_list = [64,128,256 ]
number_top_words = 100
number_top_documents = 100
additional_stop_words = ['flight', '..', 'agent', 'help', 'thank', 'thanks','great', 'need', 'response', 'today',"$", "w/",
                        "s!b" ,"x.", "it-","="]
stop_words = list(STOP_WORDS) + additional_stop_words

In [20]:
npmi = Coherence(texts = prep_dataset_for_octis(airline_tweets), topk=number_top_words, measure='c_npmi',)
npmi

In [21]:
topic_diversity = TopicDiversity(topk=number_top_words)

max_iter = 500

In [22]:
data_converted = prep_dataset_for_octis(airline_tweets)

In [23]:
def list_to_string(tweet_list):
    
    string = " ".join(tweet_list)
    
    return string

In [24]:
TfIdf_data = list(map(list_to_string,data_converted))
# TfIdf_data

In [25]:
from octis.preprocessing.preprocessing import Preprocessing,TfidfVectorizer

In [26]:
tf_idf = TfidfVectorizer()
TfIdf = tf_idf.fit(TfIdf_data)

In [27]:
# TfIdf.vocabulary_

In [28]:
TfIdf_result = tf_idf.transform(TfIdf_data)
TfIdf_result

<2913x3380 sparse matrix of type '<class 'numpy.float64'>'
	with 21925 stored elements in Compressed Sparse Row format>

In [29]:
TfIdf_matrix = TfIdf_result.toarray()
TfIdf_matrix.shape

(2913, 3380)

In [30]:
vocab = TfIdf.vocabulary_
vocab = dict([(value,key) for key,value in vocab.items()])
vocab

{3284: 'well',
 3007: 'time',
 1208: 'flight',
 1276: 'friday',
 2022: 'need',
 539: 'change',
 3329: 'worried',
 1461: 'hold',
 125: 'agent',
 272: 'available',
 2801: 'speak',
 948: 'dm',
 2295: 'possible',
 1246: 'follow',
 1262: 'fortunately',
 2825: 'staff',
 1758: 'like',
 1826: 'lynn',
 826: 'deedee',
 96: 'actually',
 3127: 'understand',
 782: 'customer',
 2683: 'service',
 2734: 'simply',
 2035: 'nice',
 1400: 'hang',
 3260: 'waste',
 1492: 'hour',
 2899: 'suppose',
 399: 'book',
 3264: 'way',
 280: 'award',
 2445: 'receive',
 1481: 'horrible',
 3061: 'treat',
 2288: 'poorly',
 2561: 'richard',
 3021: 'today',
 1093: 'excuse',
 260: 'attitude',
 169: 'amp',
 2569: 'rip',
 2998: 'ticket',
 1686: 'kill',
 1601: 'inside',
 2029: 'new',
 1812: 'love',
 56: 'aa',
 1549: 'impressed',
 2879: 'subpar',
 2255: 'plane',
 1310: 'gate',
 1978: 'move',
 843: 'delta',
 898: 'direct',
 2969: 'thank',
 83: 'accommodation',
 131: 'agree',
 1770: 'literally',
 3014: 'tixs',
 1137: 'face',
 2595

In [31]:
reconstruction_error_list = []
number_of_iterations_list = []
npmi_coherence_score_list = []
diversity_score_list = []

for number_topics in number_topic_list:
    current_number_topic = number_topics
    nmf_model = NMF(n_components=current_number_topic, init="nndsvd", max_iter=max_iter, random_state=32,
                   verbose=1).fit(TfIdf_matrix)
    nmf_W = nmf_model.transform(TfIdf_matrix)
    nmf_H = nmf_model.components_
    
    reconstruction_error_list.append(nmf_model.reconstruction_err_)
    number_of_iterations_list.append(nmf_model.n_iter_)
    octis_topic_dict = format_NMF_output(nmf_H, nmf_W, vocab, number_top_words)
    
    npmi_coherence_score_list.append(npmi.score(octis_topic_dict))
    diversity_score_list.append(topic_diversity.score(octis_topic_dict))
    

violation: 1.0
violation: 0.31944572715596736
violation: 0.224633658758665
violation: 0.13768037188748414
violation: 0.08919397020202466
violation: 0.056327347189133535
violation: 0.03930691287435603
violation: 0.028117450776816495
violation: 0.02118220302998307
violation: 0.016547870408326212
violation: 0.013446519677192747
violation: 0.01132327489928531
violation: 0.009588988281247261
violation: 0.008089509959415956
violation: 0.0070996972889035
violation: 0.006599191860497809
violation: 0.006621382375805937
violation: 0.0070289447017686585
violation: 0.007754380395012711
violation: 0.007820223346137356
violation: 0.006811970841178595
violation: 0.005509117525594734
violation: 0.0045402506827963264
violation: 0.003780139718565833
violation: 0.0033333607901949187
violation: 0.003011252728072075
violation: 0.0027471963078664124
violation: 0.002538700696459825
violation: 0.002378351680073174
violation: 0.002276310213357628
violation: 0.00219800099949542
violation: 0.0021422833043206516


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.


violation: 1.0
violation: 0.03808195282595993
violation: 0.001208273628801892
violation: 0.00013367168913383656
violation: 3.4185894002607226e-05
Converged at iteration 6


In [32]:
print("reconstruction_error_list", reconstruction_error_list)
print("number_of_iterations_list", number_of_iterations_list)
print("npmi_coherence_score_list", npmi_coherence_score_list)
print("diversity_score_list", diversity_score_list)

reconstruction_error_list [46.181259853063125, 42.21485642093671, 36.76549451303214]
number_of_iterations_list [162, 219, 500]
npmi_coherence_score_list [-0.4671010790233502, -0.4618232876034043, -0.44835967898771234]
diversity_score_list [0.3521875, 0.2421875, 0.1317578125]


In [33]:
nmf_H.shape

(256, 3380)